In [2]:
import pandas as pd
import pathlib
path = pathlib.Path("/home/ubuntu/giovanni")

In [53]:
sim_df = pd.read_csv(path/"code/2024_Chandrasekaran_NatureMethods/benchmark/output/compound_genetic_perturbation_cosine_similarity.csv")
jump_metadata = pd.read_csv(path/"code/2024_Chandrasekaran_NatureMethods/metadata/external_metadata/JUMP-Target-1_compound_metadata.tsv", sep="\t")

sim_df = sim_df.rename(columns={'Metadata_broad_sample': 'broad_sample'})

sim_df = pd.merge(
    sim_df,
    jump_metadata[['broad_sample', 'InChIKey', 'smiles']],
    on='broad_sample',
    how='left'
)

sim_df.to_csv(path/"data/compound_genetic_perturbation_cosine_similarity_inchikey.csv", index=False)

In [23]:
from datasets import load_dataset
drug_metadata =  load_dataset("tahoebio/Tahoe-100M","drug_metadata", split="train")
drug_metadata = drug_metadata.to_pandas()

Resolving data files:   0%|          | 0/3388 [00:00<?, ?it/s]

In [21]:
drug_metadata.to_pandas()

,drug,targets,moa-broad,moa-fine,human-approved,clinical-trials,gpt-notes-approval,canonical_smiles,pubchem_cid
0,Talc,None,unclear,unclear,yes,yes,Talc used in pharma and cosmetics; safety unde...,[OH-].[OH-].[O-][Si]12O[Si]3(O[Si](O1)(O[Si](O...,165411828.0
1,Bortezomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma and mantle cell ...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,387447.0
2,Ixazomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment.,B(C(CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O,25183872.0
3,Ixazomib citrate,"PSMB1, PSMB2, PSMB5",inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment as par...,B1(OC(=O)C(O1)(CC(=O)O)CC(=O)O)C(CC(C)C)NC(=O)...,56844015.0
4,Lactate (calcium),None,unclear,unclear,yes,yes,"Used in medical settings, but not specifically...",C.CC(C(=O)[O-])O.[Ca+2],168311648.0
...,...,...,...,...,...,...,...,...,...
374,Verteporfin,YAP1,inhibitor/antagonist,unclear,yes,yes,Used in photodynamic therapy for macular degen...,None,NaN
375,Quinidine (15% dihydroquinidine),KCNH2,inhibitor/antagonist,unclear,yes,yes,Approved for arrhythmias as part of quinine al...,COC1=CC2=C(C=CN=C2C=C1)[C@@H]([C@H]3C[C@@H]4CC...,441074.0
376,Canagliflozin (hemihydrate),SLC5A2,inhibitor/antagonist,Glucose transporter inhibitor,yes,yes,Approved for type 2 diabetes.,CC1=C(C=C(C=C1)[C@H]2[C@@H]([C@H]([C@@H]([C@H]...,24997615.0
377,Osimertinib (mesylate),EGFR,inhibitor/antagonist,EGFR/ERBB inhibitor,yes,yes,Approved for non-small cell lung cancer treatm...,CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=...,78357807.0


In [53]:
import pubchempy as pcp
import time

def get_compounds(inchikey, sleep_sec=0.3):
    """
    Get compound synonyms from an InChIKey using PubChemPy.
    
    Args:
        inchikey (str): InChIKey of the compound
        sleep_sec (float): Seconds to sleep to avoid rate limiting
        
    Returns:
        list: List of synonyms (empty if error or none found)
    """
    try:
        # Get compound from InChIKey
        compounds = pcp.get_compounds(inchikey, 'inchikey')
        if not compounds:
            return []
        
        # Sleep to avoid rate limiting
        time.sleep(sleep_sec)
        return compounds
        
        # # Get synonyms using CID
        # synonyms_data = pcp.get_synonyms(cid, 'cid')
        # if not synonyms_data:
        #     return compounds[0].synonyms or []
            
        # # Return synonyms list
        # return compounds synonyms_data[0].get('Synonym', [])
        
    except Exception as e:
        print(f"Error for {inchikey}: {e}")
        return None

# Example usage
inchikey = {}
for k in sim_df.InChIKey.unique():
    compounds = get_compounds(k)
    if len(compounds) > 0:
        inchikey[k] = compounds

# synonyms = get_synonyms(inchikey)

In [90]:
chemid_dict = {}
for k,v in inchikey.items():
    for c in v:
        chemid_dict[c.cid] = k

import numpy as np
pubchem_id_tahoe = drug_metadata["pubchem_cid"].values.tolist()
pubchem_id_jump = list(chemid_dict.keys())
chemid_dict = {k:v for k,v in chemid_dict.items() if k in pubchem_id_tahoe}

In [98]:
df_inchi_pubmed_id = pd.DataFrame(chemid_dict, index=["InChIKey"]).T.reset_index()
df_inchi_pubmed_id.columns = ["pubchem_cid", "InChIKey", ]

In [100]:
drug_metadata = drug_metadata.merge(
    df_inchi_pubmed_id,
    on="pubchem_cid",
    how="left"
)

drug_metadata

,drug,targets,moa-broad,moa-fine,human-approved,clinical-trials,gpt-notes-approval,canonical_smiles,pubchem_cid,InChIKey
0,Talc,None,unclear,unclear,yes,yes,Talc used in pharma and cosmetics; safety unde...,[OH-].[OH-].[O-][Si]12O[Si]3(O[Si](O1)(O[Si](O...,165411828.0,NaN
1,Bortezomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma and mantle cell ...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,387447.0,NaN
2,Ixazomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment.,B(C(CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O,25183872.0,NaN
3,Ixazomib citrate,"PSMB1, PSMB2, PSMB5",inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment as par...,B1(OC(=O)C(O1)(CC(=O)O)CC(=O)O)C(CC(C)C)NC(=O)...,56844015.0,NaN
4,Lactate (calcium),None,unclear,unclear,yes,yes,"Used in medical settings, but not specifically...",C.CC(C(=O)[O-])O.[Ca+2],168311648.0,NaN
...,...,...,...,...,...,...,...,...,...,...
374,Verteporfin,YAP1,inhibitor/antagonist,unclear,yes,yes,Used in photodynamic therapy for macular degen...,None,NaN,NaN
375,Quinidine (15% dihydroquinidine),KCNH2,inhibitor/antagonist,unclear,yes,yes,Approved for arrhythmias as part of quinine al...,COC1=CC2=C(C=CN=C2C=C1)[C@@H]([C@H]3C[C@@H]4CC...,441074.0,LOUPRKONTZGTKE-LHHVKLHASA-N
376,Canagliflozin (hemihydrate),SLC5A2,inhibitor/antagonist,Glucose transporter inhibitor,yes,yes,Approved for type 2 diabetes.,CC1=C(C=C(C=C1)[C@H]2[C@@H]([C@H]([C@@H]([C@H]...,24997615.0,NaN
377,Osimertinib (mesylate),EGFR,inhibitor/antagonist,EGFR/ERBB inhibitor,yes,yes,Approved for non-small cell lung cancer treatm...,CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=...,78357807.0,NaN


In [36]:
drug_metadata[["drug","InChIKey"]].dropna()

,drug,InChIKey
36,Salicylic acid,YGSDEFSMJLZEOE-UHFFFAOYSA-N
58,5-Azacytidine,NMUSYJAQQFHJEW-KVTDHHQDSA-N
69,Larotrectinib,NYNZQNWKBKUAII-KBXCAEBGSA-N
72,Filgotinib,RIJLVEAXPNLDTC-UHFFFAOYSA-N
75,Cilostazol,RRGUKTPIGVIEKM-UHFFFAOYSA-N
81,Lonafarnib,DHMTURDWPRKSOA-RUZDIDTESA-N
95,Acetohexamide,VGZSUPCWNCWDAN-UHFFFAOYSA-N
113,Homoharringtonine,HYFHYPWGAURHIV-JFIAXGOJSA-N
169,Ponatinib,PHXJVRSECIGDHY-UHFFFAOYSA-N
187,Nevirapine,NQDJXKOVJZTUJA-UHFFFAOYSA-N


In [6]:
import pandas as pd
drug_metadata = pd.read_csv(path/"data/drug_metadata_inchikey.csv")

In [21]:
drug_metadata.columns

Index(['drug', 'targets', 'moa-broad', 'moa-fine', 'human-approved',
       'clinical-trials', 'gpt-notes-approval', 'canonical_smiles',
       'pubchem_cid', 'InChIKey'],
      dtype='object')

In [47]:
drug = "Olanzapine"
inchikey = drug_metadata[drug_metadata.drug.isin([drug])]["InChIKey"].values[0]

In [48]:
sim_df[sim_df.InChIKey.isin([inchikey])]["Metadata_matching_target"].unique() #.sort_values(by="cosine_similarity", ascending=True)

array(['HTR2C', 'HTR3A', 'ADRA2B', 'GABRB2', 'CHRM3', 'CHRM2', 'HRH4'],
      dtype=object)

In [52]:
sim_df

,Cell,Genetic_Perturbation,Modality_1_timepoint,Modality_2_timepoint,cosine_sim,broad_sample,Metadata_matching_target,InChIKey,smiles
0,A549,CRISPR,long,long,-0.588253,BRD-A89164055-001-03-3,AKR1B1,LXANPKRCLVQAOG-UHFFFAOYSA-N,Fc1ccc2OCCC3(NC(=O)NC3=O)c2c1
1,A549,CRISPR,long,long,-0.576196,BRD-A74391928-051-03-9,CACNG1,ALOBUEHUHMBRLE-UHFFFAOYSA-N,CCCCCCCN(CC)CCCC(O)c1ccc(NS(C)(=O)=O)cc1
2,A549,CRISPR,long,long,-0.549354,BRD-K08893438-001-06-4,RGS4,QUIIIYITNGOFEI-UHFFFAOYSA-N,Cc1ccc(cc1)-n1sc(=O)n(Cc2ccc(F)cc2)c1=O
3,A549,CRISPR,long,long,-0.501251,BRD-K38512030-001-01-7,SLCO2B1,HJYYPODYNSCCOU-ODRIEIDWSA-N,CO[C@H]1\C=C\O[C@@]2(C)Oc3c(C2=O)c2c(O)cc(NC(=...
4,A549,CRISPR,long,long,-0.433312,BRD-K22482860-001-20-6,KCNH7,NUKYPUAOHBNCPY-UHFFFAOYSA-N,Nc1ccncc1
...,...,...,...,...,...,...,...,...,...
7451,U2OS,ORF,short,short,0.616746,BRD-K64890080-001-02-1,BRD4,XQVVPGYIWAGRNI-JOCHJYFZSA-N,CC[C@H]1N(C2CCCC2)c2nc(Nc3ccc(cc3OC)C(=O)NC3CC...
7452,U2OS,ORF,short,short,0.644838,BRD-K93779381-001-01-9,PRKCE,VDJHFHXMUKFKET-WDUFCVPESA-N,C\C=C(\C)C(=O)O[C@H]1C(C)=C[C@@]23[C@H](C)C[C@...
7453,U2OS,ORF,short,short,0.701634,BRD-K44227013-001-08-0,LYN,PHXJVRSECIGDHY-UHFFFAOYSA-N,CN1CCN(Cc2ccc(NC(=O)c3ccc(C)c(c3)C#Cc3cnc4cccn...
7454,U2OS,ORF,short,short,0.702138,BRD-K95785537-001-22-3,ABL1,PBBRWFOVCUAONR-UHFFFAOYSA-N,CC(C)(C)n1nc(-c2ccc(Cl)cc2)c2c(N)ncnc12


In [50]:
jump_path = pathlib.Path("/home/ubuntu/giovanni")
pd.read_csv(jump_path/"data/drug_metadata_inchikey.csv")

,drug,targets,moa-broad,moa-fine,human-approved,clinical-trials,gpt-notes-approval,canonical_smiles,pubchem_cid,InChIKey
0,Talc,NaN,unclear,unclear,yes,yes,Talc used in pharma and cosmetics; safety unde...,[OH-].[OH-].[O-][Si]12O[Si]3(O[Si](O1)(O[Si](O...,165411828.0,NaN
1,Bortezomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma and mantle cell ...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,387447.0,NaN
2,Ixazomib,PSMB5,inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment.,B(C(CC(C)C)NC(=O)CNC(=O)C1=C(C=CC(=C1)Cl)Cl)(O)O,25183872.0,NaN
3,Ixazomib citrate,"PSMB1, PSMB2, PSMB5",inhibitor/antagonist,Proteasome inhibitor,yes,yes,Approved for multiple myeloma treatment as par...,B1(OC(=O)C(O1)(CC(=O)O)CC(=O)O)C(CC(C)C)NC(=O)...,56844015.0,NaN
4,Lactate (calcium),NaN,unclear,unclear,yes,yes,"Used in medical settings, but not specifically...",C.CC(C(=O)[O-])O.[Ca+2],168311648.0,NaN
...,...,...,...,...,...,...,...,...,...,...
374,Verteporfin,YAP1,inhibitor/antagonist,unclear,yes,yes,Used in photodynamic therapy for macular degen...,NaN,NaN,NaN
375,Quinidine (15% dihydroquinidine),KCNH2,inhibitor/antagonist,unclear,yes,yes,Approved for arrhythmias as part of quinine al...,COC1=CC2=C(C=CN=C2C=C1)[C@@H]([C@H]3C[C@@H]4CC...,441074.0,LOUPRKONTZGTKE-LHHVKLHASA-N
376,Canagliflozin (hemihydrate),SLC5A2,inhibitor/antagonist,Glucose transporter inhibitor,yes,yes,Approved for type 2 diabetes.,CC1=C(C=C(C=C1)[C@H]2[C@@H]([C@H]([C@@H]([C@H]...,24997615.0,NaN
377,Osimertinib (mesylate),EGFR,inhibitor/antagonist,EGFR/ERBB inhibitor,yes,yes,Approved for non-small cell lung cancer treatm...,CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=...,78357807.0,NaN


In [51]:
pd.read_csv(jump_path/"code/2024_Chandrasekaran_NatureMethods/benchmark/output/compound_genetic_perturbation_cosine_similarity.csv")

,Cell,Genetic_Perturbation,Modality_1_timepoint,Modality_2_timepoint,cosine_sim,Metadata_broad_sample,Metadata_matching_target
0,A549,CRISPR,long,long,-0.588253,BRD-A89164055-001-03-3,AKR1B1
1,A549,CRISPR,long,long,-0.576196,BRD-A74391928-051-03-9,CACNG1
2,A549,CRISPR,long,long,-0.549354,BRD-K08893438-001-06-4,RGS4
3,A549,CRISPR,long,long,-0.501251,BRD-K38512030-001-01-7,SLCO2B1
4,A549,CRISPR,long,long,-0.433312,BRD-K22482860-001-20-6,KCNH7
...,...,...,...,...,...,...,...
7451,U2OS,ORF,short,short,0.616746,BRD-K64890080-001-02-1,BRD4
7452,U2OS,ORF,short,short,0.644838,BRD-K93779381-001-01-9,PRKCE
7453,U2OS,ORF,short,short,0.701634,BRD-K44227013-001-08-0,LYN
7454,U2OS,ORF,short,short,0.702138,BRD-K95785537-001-22-3,ABL1
